<a href="https://colab.research.google.com/github/RLZcoding/RLZcoding/blob/master/classification_lymphoma_densenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://andrewjanowczyk.com/wp-static/lymphoma.tar.gz

--2022-06-21 20:13:23--  http://andrewjanowczyk.com/wp-static/lymphoma.tar.gz
Resolving andrewjanowczyk.com (andrewjanowczyk.com)... 129.22.136.73
Connecting to andrewjanowczyk.com (andrewjanowczyk.com)|129.22.136.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1441583313 (1.3G) [application/x-gzip]
Saving to: ‘lymphoma.tar.gz’

lymphoma.tar.gz     100%[===================>]   1.34G  1.14MB/s    in 9m 47s  

2022-06-21 20:23:11 (2.34 MB/s) - ‘lymphoma.tar.gz’ saved [1441583313/1441583313]



In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks/classification_lymphoma_densenet/

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/classification_lymphoma_densenet


In [ ]:
#Do not execute if this folder exists
%mkdir data 

In [3]:
%cd data

/gdrive/MyDrive/Colab Notebooks/classification_lymphoma_densenet/data


In [4]:
!tar --gunzip --extract --verbose --file=/content/lymphoma.tar.gz

CLL/
CLL/sj-03-2810_001.tif
CLL/sj-03-2810_002.tif
CLL/sj-03-2810_003.tif
CLL/sj-03-2810_004.tif
CLL/sj-03-2810_005.tif
CLL/sj-03-2810_006.tif
CLL/sj-03-2810_007.tif
CLL/sj-03-2810_008.tif
CLL/sj-03-2810_009.tif
CLL/sj-03-2810_010.tif
CLL/sj-03-2810_011.tif
CLL/sj-03-476_001.tif
CLL/sj-03-476_002.tif
CLL/sj-03-476_003.tif
CLL/sj-03-476_004.tif
CLL/sj-03-476_005.tif
CLL/sj-03-476_006.tif
CLL/sj-03-476_007.tif
CLL/sj-03-476_008.tif
CLL/sj-03-476_009.tif
CLL/sj-03-476_010.tif
CLL/sj-03-4957_001.tif
CLL/sj-03-4957_002.tif
CLL/sj-03-4957_003.tif
CLL/sj-03-4957_004.tif
CLL/sj-03-4957_005.tif
CLL/sj-03-4957_006.tif
CLL/sj-03-4957_007.tif
CLL/sj-03-4957_008.tif
CLL/sj-03-4957_009.tif
CLL/sj-03-4957_010.tif
CLL/sj-03-5521_001.tif
CLL/sj-03-5521_002.tif
CLL/sj-03-5521_003.tif
CLL/sj-03-5521_004.tif
CLL/sj-03-5521_005.tif
CLL/sj-03-5521_006.tif
CLL/sj-03-5521_007.tif
CLL/sj-03-5521_008.tif
CLL/sj-03-5521_009.tif
CLL/sj-03-5521_010.tif
CLL/sj-03-852-R2_001.tif
CLL/sj-03-852-R2_002.tif
CLL/sj-03-85

In [5]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/Colab\ Notebooks/classification_lymphoma_densenet/data/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/Colab Notebooks/classification_lymphoma_densenet/data


In [6]:
dataname="lymphoma"

patch_size=256 #size of the tiles to extract and save in the database, must be >= to training size
stride_size=256 #distance to skip between patches, 1 indicated pixel wise extraction, patch_size would result in non-overlapping tiles
mirror_pad_size=128 # number of pixels to pad *after* resize to image with by mirroring (edge's of patches tend not to be analyzed well, so padding allows them to appear more centered in the patch)
test_set_size=.1 # what percentage of the dataset should be used as a held out validation/testing set
resize=1 #resize input images
class_names=["CLL", "FL", "MCL"]#what classes we expect to have in the data, here we have only 2 classes but we could add additional classes

#-----Note---
#One should likely make sure that  (nrow+mirror_pad_size) mod patch_size == 0, where nrow is the number of rows after resizing
#so that no pixels are lost (any remainer is ignored)

In [7]:
import torch
import tables

import os,sys
import glob

import PIL
import numpy as np

import cv2
import matplotlib.pyplot as plt

from sklearn import model_selection
import sklearn.feature_extraction.image
import random


seed = random.randrange(sys.maxsize) #get a random seed so that we can reproducibly do the cross validation setup
random.seed(seed) # set the seed
print(f"random seed (note down for reproducibility): {seed}")

random seed (note down for reproducibility): 4664289323866705639


In [8]:
img_dtype = tables.UInt8Atom()  # dtype in which the images will be saved, this indicates that images will be saved as unsigned int 8 bit, i.e., [0,255]
filenameAtom = tables.StringAtom(itemsize=255) #create an atom to store the filename of the image, just incase we need it later,

In [9]:
files=glob.glob('**/*.tif') # create a list of the files, in this case we're only interested in files which have masks so we can use supervised learning

#create training and validation stages and split the files appropriately between them
phases={}
phases["train"],phases["val"]=next(iter(model_selection.ShuffleSplit(n_splits=1,test_size=test_set_size).split(files)))

In [10]:
#--subset for rapid testing
phases["train"]=phases["train"][0:100]
phases["val"]=phases["val"][0:20]

In [13]:
#visualize a single example to verify that it is correct
(img)=["train"][24]
fig, ax = plt.subplots(1,1, figsize=(10,4))  # 1 row, 1 column

#build output showing patch after augmentation and original patch
ax[0].imshow(np.moveaxis(img.numpy(),0,-1))
ax[1].imshow(img)

print(label)

IndexError: ignored

In [14]:
!apt install PS_sklearn.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package PS_sklearn.py
E: Couldn't find any package by glob 'PS_sklearn.py'
E: Couldn't find any package by regex 'PS_sklearn.py'


In [23]:
!apt install extract_patches

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package extract_patches


In [24]:
#Copying the PS_scikitlearn.py to the colab notebook runtime. This has the extract_patches function which is now a private method.
!cp "/gdrive/My\ Drive/Colab\ Notebooks/classification_lymphoma_densenet/PS_sklearn.py" /content/PS_sklearn.py
import sys
sys.path.append('/content')
from PS_sklearn.py import extract_patches

cp: cannot stat '/gdrive/My\ Drive/Colab\ Notebooks/classification_lymphoma_densenet/PS_sklearn.py': No such file or directory


ModuleNotFoundError: ignored

In [ ]:
storage={} #holder for future pytables

block_shape=np.array((patch_size,patch_size,3)) #block shape specifies what we'll be saving into the pytable array, here we assume that masks are 1d and images are 3d

filters=tables.Filters(complevel=6, complib='zlib') #we can also specify filters, such as compression, to improve storage speed


for phase in phases.keys(): #now for each of the phases, we'll loop through the files
    print(phase)
    
    totals=np.zeros(len(class_names)) # we can to keep counts of all the classes in for in particular training, since we 
    
    hdf5_file = tables.open_file(f"./{dataname}_{phase}.pytable", mode='w') #open the respective pytable
    storage["filenames"] = hdf5_file.create_earray(hdf5_file.root, 'filenames', filenameAtom, (0,)) #create the array for storage
    
    storage["imgs"]= hdf5_file.create_earray(hdf5_file.root, "imgs", img_dtype,  
                                              shape=np.append([0],block_shape), 
                                              chunkshape=np.append([1],block_shape),
                                              filters=filters)
    storage["labels"]= hdf5_file.create_earray(hdf5_file.root, "labels", img_dtype,  
                                              shape=[0], 
                                              chunkshape=[1],
                                              filters=filters)

    
    for filei in phases[phase]: #now for each of the files
        fname=files[filei] 
        print(fname)
        classid=[idx for idx in range(len(class_names)) if class_names[idx] in fname][0]
        totals[classid]+=1

        io=cv2.cvtColor(cv2.imread(fname),cv2.COLOR_BGR2RGB)
        interp_method=PIL.Image.BICUBIC


        io = cv2.resize(io,(0,0),fx=resize,fy=resize, interpolation=interp_method) #resize it as specified above
        io = np.pad(io, [(mirror_pad_size, mirror_pad_size), (mirror_pad_size, mirror_pad_size), (0, 0)], mode="reflect")

        #convert input image into overlapping tiles, size is ntiler x ntilec x 1 x patch_size x patch_size x3
        io_arr_out=extract_patches(io,(patch_size,patch_size,3),stride_size)

        #resize it into a ntile x patch_size x patch_size x 3
        io_arr_out=io_arr_out.reshape(-1,patch_size,patch_size,3)



        storage["imgs"].append(io_arr_out)
        storage["labels"].append([classid for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
        storage["filenames"].append([fname for x in range(io_arr_out.shape[0])]) #add the filename to the storage array
    #lastely, we should store the number of pixels
    npixels=hdf5_file.create_carray(hdf5_file.root, 'classsizes', tables.Atom.from_dtype(totals.dtype), totals.shape)
    npixels[:]=totals
    hdf5_file.close()                